In [ ]:
import os
from copy import deepcopy

import numpy as np

from mcbn.utils.helper import get_setup
from mcbn.utils.helper import dump_yaml
from mcbn.utils.helper import get_directories_in_path
from mcbn.utils.helper import get_logger
from mcbn.environment.constants import TAU_EVAL_PATH

In [ ]:
logger = get_logger()

logger.info("STEP 5: Get best TAU results")

s = get_setup()

In [ ]:
# Get evaluations performed in order from last evaluation to first
evaluation_dirs = sorted(os.listdir(TAU_EVAL_PATH), reverse=True)

# Keep track of parsed datasets
parsed_datasets = []

# Add 'const' variants to optimized models
optimized_models = deepcopy(s['models'])
optimized_models += [m + ' const' for m in optimized_models if 'MC' in m]

# Store all results in a dict
results = {}

# Iterate over all evaluation dirs
for eval_dir in evaluation_dirs:
    
    # Get all dataset-specific subdirs in evaluation dir
    eval_path = os.path.join(TAU_EVAL_PATH, eval_dir)
    dataset_dirs = get_directories_in_path(eval_path)
    
    # Iterate over dataset-specific subdirs
    for dataset_name in dataset_dirs:
        
        # Make sure we have not added a later evaluation of this dataset to results
        if not dataset_name in parsed_datasets:
            
            dataset_eval_path = os.path.join(eval_path, dataset_name)
            results[dataset_name] = {}
            
            # Iterate over optimized models
            for model in optimized_models:
                
                # Get optimized tau
                opt_result_path = os.path.join(dataset_eval_path, model + '_opt.txt')
                tau = np.loadtxt(opt_result_path, dtype=float).tolist()
                
                results[dataset_name][model] = tau
            
            parsed_datasets.append(dataset_name)

dump_yaml(results, os.getcwd(), 'tau_results.yml')
logger.info("DONE STEP 5")